# Merge station data from the 4D var and verification runs

In [2]:
import dask
dask.config.set(scheduler='processes')


In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from datetime import datetime
from scipy.interpolate import PchipInterpolator

from glob import glob
from matplotlib import rcParams
from soda.dataio.roms.romsio import  get_depth
from soda.utils.myproj import MyProj
from mycurrents import oceanmooring as om
#rcParams['font.family'] = 'sans-serif'
#rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
#rcParams['font.serif'] = ['Bitstream Vera Sans']
rcParams["font.size"] = "12"
rcParams['axes.labelsize']='medium'


Bad key "backend.qt5" on line 5 in
/home/mrayson/.config/matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
/home/mrayson/code/soda/soda/dataio/netcdfio.py:40: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  ncmeta = yaml.load(f)


Warning - could not import octant package.


In [16]:
!ls /scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/verification


archive_fwd_ver_6241.nc			  log_ver_6277.txt  qck_ver_6261.nc
archive_fwd_ver_6245.nc			  log_ver_6281.txt  qck_ver_6265.nc
archive_fwd_ver_6249.nc			  log_ver_6285.txt  qck_ver_6269.nc
archive_fwd_ver_6253.nc			  log_ver_6289.txt  qck_ver_6273.nc
archive_fwd_ver_6257.nc			  log_ver_6293.txt  qck_ver_6277.nc
archive_fwd_ver_6261.nc			  log_ver_6297.txt  qck_ver_6281.nc
archive_fwd_ver_6265.nc			  log_ver_6301.txt  qck_ver_6285.nc
archive_fwd_ver_6269.nc			  log_ver_6305.txt  qck_ver_6289.nc
archive_fwd_ver_6273.nc			  log_ver_6309.txt  qck_ver_6293.nc
archive_fwd_ver_6277.nc			  log_ver_6313.txt  qck_ver_6297.nc
archive_fwd_ver_6281.nc			  log_ver_6317.txt  qck_ver_6301.nc
archive_fwd_ver_6285.nc			  log_ver_6321.txt  qck_ver_6305.nc
archive_fwd_ver_6289.nc			  log_ver_6325.txt  qck_ver_6309.nc
archive_fwd_ver_6293.nc			  log_ver_6329.txt  qck_ver_6313.nc
archive_fwd_ver_6297.nc			  log_ver_6333.txt  qck_ver_6317.nc
archive_fwd_ver_6301.nc			  log_ver_6337.txt  qck_ver_6321.nc
archive_

In [24]:
# Load a ROMS timeseries object
def get_roms_station_da(romsfile, romsvar, xyin, cycle=0, ncycles=3):
    dsroms = xr.open_dataset(romsfile)
    #print(dsroms)
    zroms = get_depth(dsroms.s_rho.values, dsroms.Cs_r.values, dsroms.hc.values, dsroms.h.values)# , \
    #         Vtransform=dsroms.Vtransform.values)
    xroms = dsroms.lon_rho.values
    yroms = dsroms.lat_rho.values
    
    nt = dsroms.ocean_time.size//ncycles

    # Get the point
    dist = np.abs( (xyin[0]-xroms) + 1j*(xyin[1]-yroms))
    idx = np.argwhere(dist==dist.min())[0,0]
    #print(xyin, xroms[idx], yroms[idx], dist[idx], zroms[:,idx])
    print(xyin[0], xroms[idx], xyin[1], yroms[idx], dist[idx])
    
    t0 = cycle*nt
    t1 = t0 + nt

    data = dsroms[romsvar][t0:t1,idx,:].values
    #print(zroms[...].shape, data.shape)
    #print(zroms.ravel())

    #Fi = interp1d(zroms[...,idx].squeeze(), data.squeeze(), axis=1, fill_value='extrapolate')
    
    #return om.OceanMooring(dsroms.ocean_time.values[t0:t1], Fi(zin), zin)
    return om.OceanMooring(dsroms.ocean_time.values[t0:t1], data.squeeze(), zroms[...,idx].squeeze())

def get_roms_station(romsfile, romsvar, xyin):
    dsroms = xr.open_dataset(romsfile)
    #print(dsroms)
    zroms = get_depth(dsroms.s_rho.values, dsroms.Cs_r.values, dsroms.hc.values, dsroms.h.values)# , \
    #         Vtransform=dsroms.Vtransform.values)
    xroms = dsroms.lon_rho.values
    yroms = dsroms.lat_rho.values

    # Get the point
    dist = np.abs( (xyin[0]-xroms) + 1j*(xyin[1]-yroms))
    idx = np.argwhere(dist==dist.min())[0,0]
    #print(xyin, xroms[idx], yroms[idx])

    data = dsroms[romsvar][:,idx,:]
    return om.OceanMooring(dsroms.ocean_time.values, data.squeeze(), zroms[...,idx].squeeze())
    #print(zroms[...].shape, data.shape)
    #print(zroms.ravel())
    
    #Fi = interp1d(zroms[...,idx].squeeze(), data.squeeze(), axis=1, fill_value='extrapolate')
    
    #return om.OceanMooring(dsroms.ocean_time.values, Fi(zin), zin)

In [10]:
# Add the site location
P = MyProj(None, utmzone=51,isnorth=False)

lon_prelude, lat_prelude = P.to_ll(534322.7, 8475878.4)

sites = pd.DataFrame(np.array([[lon_prelude, 123.3535, 123.2803, 123.346383,127.56335, 123.0459],\
            [lat_prelude, -13.7621, -13.8170, -13.75895, -9.8122, -14.2426]]).T,
            index=['Prelude','NP250','DWR','KP150_phs2','ITFTIS','BRB200'], columns=['lon','lat'])


In [17]:
roms4dvarfiles = sorted(glob('/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/sta_6*.nc'))
romsfiles = sorted(glob('/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/verification/sta_ver_6*.nc'))

In [18]:
romsfiles[0:2],roms4dvarfiles[0:2]

(['/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/verification/sta_ver_6241.nc',
  '/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/verification/sta_ver_6245.nc'],
 ['/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/sta_6241.nc',
  '/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/sta_6253.nc'])

In [19]:
def load_station(loadfunc, roms4dvarfiles, sitename, varname):
    xyin = [sites['lon'][sitename],sites['lat'][sitename] ]

    ds = get_roms_station_da(roms4dvarfiles[0], varname, xyin)
    for ff in roms4dvarfiles[1:]:
        ds = ds.concat(get_roms_station_da(ff, varname, xyin))

    #ds.long_name = 'Water Temperature'
    #ds.units = 'degrees C'
    ds.X = xyin[0]
    ds.Y = xyin[1]
    ds.StationName = sitename
    return ds.to_xray()

In [73]:
sitename = 'NP250'
outfile = '../DATA/ROMS_4DVAR_station_{}.nc'.format(sitename)
T = load_station(get_roms_station_da, roms4dvarfiles, sitename,'temp')
u = load_station(get_roms_station_da, roms4dvarfiles, sitename,'u_eastward')
v = load_station(get_roms_station_da, roms4dvarfiles, sitename,'v_northward')
xr.Dataset({'temp':T,'u_eastward':u,'v_northward':v}).to_netcdf(outfile)
print(outfile)

outfile = '../DATA/ROMS_NL_station_{}.nc'.format(sitename)
T = load_station(get_roms_station, romsfiles, sitename,'temp')
u = load_station(get_roms_station, romsfiles, sitename,'u_eastward')
v = load_station(get_roms_station, romsfiles, sitename,'v_northward')
xr.Dataset({'temp':T,'u_eastward':u,'v_northward':v}).to_netcdf(outfile)
print(outfile)

../DATA/ROMS_4DVAR_station_NP250.nc
../DATA/ROMS_NL_station_NP250.nc


In [74]:
sitename = 'ITFTIS'
outfile = '../DATA/ROMS_4DVAR_station_{}.nc'.format(sitename)
T = load_station(get_roms_station_da, roms4dvarfiles, sitename,'temp')
u = load_station(get_roms_station_da, roms4dvarfiles, sitename,'u_eastward')
v = load_station(get_roms_station_da, roms4dvarfiles, sitename,'v_northward')
xr.Dataset({'temp':T,'u_eastward':u,'v_northward':v}).to_netcdf(outfile)
print(outfile)

outfile = '../DATA/ROMS_NL_station_{}.nc'.format(sitename)
T = load_station(get_roms_station, romsfiles, sitename,'temp')
u = load_station(get_roms_station, romsfiles, sitename,'u_eastward')
v = load_station(get_roms_station, romsfiles, sitename,'v_northward')
xr.Dataset({'temp':T,'u_eastward':u,'v_northward':v}).to_netcdf(outfile)
print(outfile)

../DATA/ROMS_4DVAR_station_ITFTIS.nc
../DATA/ROMS_NL_station_ITFTIS.nc


In [25]:
sitename = 'BRB200'
outfile = '../DATA/ROMS_4DVAR_station_{}.nc'.format(sitename)
T = load_station(get_roms_station_da, roms4dvarfiles, sitename,'temp')
u = load_station(get_roms_station_da, roms4dvarfiles, sitename,'u_eastward')
v = load_station(get_roms_station_da, roms4dvarfiles, sitename,'v_northward')
xr.Dataset({'temp':T,'u_eastward':u,'v_northward':v}).to_netcdf(outfile)
print(outfile)

outfile = '../DATA/ROMS_NL_station_{}.nc'.format(sitename)
T = load_station(get_roms_station, romsfiles, sitename,'temp')
u = load_station(get_roms_station, romsfiles, sitename,'u_eastward')
v = load_station(get_roms_station, romsfiles, sitename,'v_northward')
xr.Dataset({'temp':T,'u_eastward':u,'v_northward':v}).to_netcdf(outfile)
print(outfile)

123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.24260352959

123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.242603529598123 4.132172328165077e-06
123.0459 123.04590214867058 -14.2426 -14.24260352959